<img src="q.png" width="600" height="500">


# Datos personales


### Apellidos: Alarcón Tamayo

### Nombre: Alfonso

In [ ]:
# Importaciones
from itertools import product
from itertools import combinations

import random
import math
import copy  
import time

# S

In [ ]:
# Generamos el conjunto S con las restricciones especificadas para el apartado A, aunque es valido para cualquiera
def generar_S(resultados_posibles=[0,1,2],num_triples = 6, min_1=0, max_1=2, min_2=0, max_2=2, sum_1_2=[2, 3, 4]):
    
    # Generamos las combinaciones posibles y creamos la solución vacia donde guardaremos las del espacio acotado
    combinaciones = list(product(resultados_posibles, repeat=num_triples))
    combinaciones_filtradas = []
    
    # Para cada combinacion comprobamos las restricciones, si las cumple pues la guardamos
    for combinacion in combinaciones:
        # Comprobamos si el número de ocurrencias de 1 y 2 está dentro del rango permitido (min_1 y max_1)
        if min_1 <= combinacion.count(1) <= max_1 and min_2 <= combinacion.count(2) <= max_2:
            suma_1_2 = combinacion.count(1) + combinacion.count(2)
            # Comprobamos si la cumple con los valores permitidos (sum_1_2)
            if suma_1_2 in sum_1_2:
                # Si cumple con todas las restricciones, la añadimos
                combinaciones_filtradas.append(combinacion)

    return combinaciones_filtradas

In [ ]:
# LLamamos al método con los valores del aparado a, aunque por defecto ya los tiene

# Así comprobamos que obtenemos los 270 Elementos igual que el enunciado
S = generar_S(resultados_posibles=[0,1,2],num_triples = 6, min_1=0, max_1=2, min_2=0, max_2=2, sum_1_2=[2, 3, 4])
print("Para el apartado a se han encontrado :", len(S))

# C

Al principio no lo tuve en cuenta, pero me di cuenta que en la explicación del profesor Victor. Comentaba que las apuestas podían no estar en C (su nucleo) y no lo tuve en cuenta, por eso me he creado este metodo. Si solo se contemplasen bolas del conjunto S, sería tan sencillo como pasarle al Generar_C directamente S y no este método auxiliar S_mas_distancia_1

In [ ]:
def S_con_distancia_mas_1(S):
    
    # Me hago una copia de S
    completo_con_distancia_1 = list(S)

    # recorro las combinaciones
    for combinacion in S:
        for i in range(len(combinacion)):
            # Recorro cada posible resultado (0, 1, 2) dependiendo del resultado posible
            for resultado in range(3):
                if combinacion[i] != resultado:
                    #creo una apuesta
                    nueva_combinacion = combinacion[:i] + (resultado,) + combinacion[i+1:]
                    # si no estaba la añado
                    if nueva_combinacion not in completo_con_distancia_1:
                        completo_con_distancia_1.append(nueva_combinacion)

    return completo_con_distancia_1


S_completo_con_distancia_1 = S_con_distancia_mas_1(S)

In [ ]:
def generarC(combinaciones_posibles, n):
    # seleccionamos n apuestas aleatorias 
    n_aleatorios = random.sample(combinaciones_posibles, n)
    #Guardo la solución aqui
    C = []
    
    for combinacion in n_aleatorios:
        C.append(combinacion)

    return C

In [ ]:
# Ejemplo de uso:
n = 150
S_con_distancia_mas_1= S_con_distancia_mas_1(S)
C = generarC(S_completo_con_distancia_1, n)

# N

In [ ]:
def calcular_N(S, C):
    # Función auxiliar para verificar si una combinación está cubierta por alguna en C
    def esta_cubierta(combinacion, conjunto_C):
        for c in conjunto_C:
            distancia = sum(1 for x, y in zip(combinacion, c) if x != y)
            if distancia <= 1:
                return True
        return False

    # Obtener las combinaciones en S que no están cubiertas por C
    apuestas_no_cubiertas = [s for s in S if not esta_cubierta(s, C)]
    
    # Contar el número de combinaciones en S que no están cubiertas por C
    cantidad_no_cubiertas = len(apuestas_no_cubiertas)
    
    # devolvemos una tupla de dos valores las apuestas del espacio sin cubrir y esas apuestas
    return cantidad_no_cubiertas, apuestas_no_cubiertas

N = calcular_N(S, C)
print("Número de apuestas no cubiertas:", N)

# Enfriamiento acero

In [ ]:
def enfriamiento_acero(C, N, R, T, factor_enfriamiento):
    
    # Registramos el tiempo inicial
    start_time = time.time()
    # Condición de parada
    while ((N[0]) > 0):  # La condición de parada es cuando N llega a 0
        
        # Comprobador de si se realiza en un tiempo aceptable, máximo 80 segundos,
        if time.time() - start_time > 80:  
            # Si se pasa del tiempo se para
            print("Tiempo máximo alcanzado (80 segundos). Terminando el método.")
            break
        
        # recorremos C
        for i in range(len(C)):

            for j in range(2):  # 2 por defecto según la teoría
                
                # Seleccionar aleatoriamente una posición en C[i]
                posicion = random.randrange(len(C[i]))
                # Generar un nuevo valor que varie en 1 respecto al valor original en la posición seleccionada
                nuevo_valor = (C[i][posicion] + random.choice([-1, 1])) % 3
                # Convertir la tupla C[i] a lista, actualizar el valor y luego convertir de nuevo a tupla
                lista_C_i = list(C[i])
                lista_C_i[posicion] = nuevo_valor
                d = tuple(lista_C_i)
                
                # Me creo una copia de C por seguridad y compruebo el si con el cambio se cubre el espacio
                C_copia = copy.deepcopy(C)
                # cambio en la pos i la apuesta por d
                C_copia[i] = d  
                # calculamos el nuevo N
                NN = calcular_N(S,C_copia)

                # restriccion 1 : Comprobar si mejora
                r1 = NN[0] <= (N[0])
                # calculamos el incremento
                incremento = abs(NN[0] - (N[0]))
                # restriccion 2:calculo 
                r2 = random.random() < (math.exp(-(incremento) / T))
                # Si se cumple una de las 2
                if ( r1 or r2 ):
                    # Sustituimos C y N
                    C[i]=d
                    N= NN

                    if(N[0] == 0):
                        print("==La Solucion encontrada ==")              
                        print("Tiempo total de ejecución:", time.time() - start_time, "segundos")
                        
                        # Salimos del bucle si se cubren todos los casos y devolvemos la solución
                        return C  
                            
        # Actualizamos la temperatura según el factor de enfriamiento
        T *= factor_enfriamiento
        
    return C

## Un ejemplo para usar el método

Tras ejecutar todo lo anterior especificar los parámetros que se quiera en los siguientes dos apartados

In [ ]:
S = generar_S(resultados_posibles=[0,1,2],num_triples = 6, min_1=0, max_1=2, min_2=0, max_2=2, sum_1_2=[2, 3, 4])
print("Para el apartado a se han encontrado :", len(S))
# Ejemplo de uso:
n = 50
C = generarC(S_completo_con_distancia_1, n)
N = calcular_N(S, C)
print("Número de apuestas no cubiertas:", N)

In [ ]:
# Ejemplo de uso:
R = 1   # Radio de las bolas
T = 2              # Temperatura inicial
factor_enfriamiento = 0.95

resultado = enfriamiento_acero(C, N, R, T, factor_enfriamiento)
print(resultado)
# Buscar
#(0, [])

### Para comprobar si cubre todo el conjunto

In [ ]:
# Nos devuelve las apuestas sin cubrir, sidevuelve 0 , el resultado es correcto
calcular_N(S,resultado)